# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [4]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [1]:
import autogen
autogen.__version__

'0.9.1post0'

# Agent Orchestration: Coordinating Multiple Agents
---

# Example: Enhancing Our Financial Compliance System with Group Chat

## The Challenge
### In our Human in the Loop example, we built a finance_bot that could process transactions and get human approval for suspicious ones. However, we now need professional, formatted summary reports of all the transactions.

## Our Group Chat Solution
### Here's how we'll enhance our system:

- ### Keep the finance_bot focused on transaction processing and human approval
- ### Create a new summary_bot specialized in generating formatted transaction reports
- ### Use the group chat with AutoPattern pattern to automatically transition from finance_bot to summary_bot when all transactions are processed
- ### Maintain human oversight for suspicious transaction approval and to terminate the conversation

## Creating the Necessary Agents

In [40]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

import os
import random

# Note: Make sure to set your API key in your environment first

# Configure the LLM
llm_config = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    temperature=0.2,
)

# Define the system message for our finance bot
finance_system_message = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.
For each transaction:
- If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval.
- Otherwise, approve it automatically.
Provide the full set of transactions to approve at one time.
If the human gives a general approval, it applies to all transactions requiring approval.
When all transactions are processed, summarize the results and say "You can type exit to finish".
"""

# Define the system message for the summary agent
summary_system_message = """
You are a financial summary assistant. You will be given a set of transaction details and their approval status.
Your task is to summarize the results of the transactions processed by the finance bot.
Generate a markdown table with the following columns:
- Vendor
- Memo
- Amount
- Status (Approved/Rejected)
The summary should include the total number of transactions, the number of approved transactions, and the number of rejected transactions.
The summary should be concise and clear.
"""

# Create the finance agent with LLM intelligence
with llm_config:
    finance_bot = ConversableAgent(
        name="finance_bot",
        system_message=finance_system_message,
    )
    summary_bot = ConversableAgent(
        name="summary_bot",
        system_message=summary_system_message,
    )

# Create the human agent for oversight
human = ConversableAgent(
    name="human",
    human_input_mode="ALWAYS",  # Always ask for human input
)

## Initiating the Group Chat

In [41]:
# Generate sample transactions - this creates different transactions each time you run
VENDORS = ["Staples", "Acme Corp", "CyberSins Ltd", "Initech", "Globex", "Unicorn LLC"]
MEMOS = ["Quarterly supplies", "Confidential", "NDA services", "Routine payment", "Urgent request", "Reimbursement"]

def generate_transaction():
    amount = random.choice([500, 1500, 9999, 12000, 23000, 4000])
    vendor = random.choice(VENDORS)
    memo = random.choice(MEMOS)
    return f"Transaction: ${amount} to {vendor}. Memo: {memo}."

# Generate 3 random transactions
transactions = [generate_transaction() for _ in range(3)]

# Format the initial message
initial_prompt = (
    "Please process the following transactions one at a time:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions)])
)

# Create pattern for the group chat
pattern = AutoPattern(
    initial_agent=finance_bot,                   # Start with the finance bot
    agents=[finance_bot, summary_bot],           # All agents in the group chat
    user_agent=human,                            # Provide our human-in-the-loop agent
    group_manager_args={"llm_config": llm_config}  # Config for group manager
)

# Initialize the group chat
result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,                     # Initial request with transactions
)

human (to chat_manager):

Please process the following transactions one at a time:

1. Transaction: $1500 to Initech. Memo: Quarterly supplies.
2. Transaction: $23000 to Staples. Memo: Routine payment.
3. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment.

--------------------------------------------------------------------------------

Next speaker: finance_bot


>>>>>>>> USING AUTO REPLY...
finance_bot (to chat_manager):

Let's process the transactions:

1. Transaction: $1500 to Initech. Memo: Quarterly supplies. 
   - This transaction is approved automatically.

2. Transaction: $23000 to Staples. Memo: Routine payment.
   - This transaction seems suspicious due to the amount exceeding $10,000. I need your approval.

3. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment.
   - This transaction also seems suspicious due to the amount exceeding $10,000. I need your approval.

Please provide your approval for the transactions requiring it, or a general approval for all

Replying as human. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  approve


human (to chat_manager):

approve

--------------------------------------------------------------------------------

Next speaker: finance_bot


>>>>>>>> USING AUTO REPLY...
finance_bot (to chat_manager):

Thank you for your approval. Here are the results of the transactions processed:

1. Transaction: $1500 to Initech. Memo: Quarterly supplies. - Approved
2. Transaction: $23000 to Staples. Memo: Routine payment. - Approved
3. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment. - Approved

All transactions have been processed successfully. You can type exit to finish.

--------------------------------------------------------------------------------

Next speaker: summary_bot


>>>>>>>> USING AUTO REPLY...
summary_bot (to chat_manager):

Here is the summary of the transactions processed:

| Vendor          | Memo                  | Amount  | Status   |
|-----------------|-----------------------|---------|----------|
| Initech         | Quarterly supplies     | $1500   | Approve

Replying as human. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (21a80879-7c3f-490c-92c8-320f21e4e2ac): User requested to end the conversation

>>>>>>>> TERMINATING RUN (46a00760-a2f0-46d4-9a83-cdc3f9b06885): No reply generated
